<a href="https://colab.research.google.com/github/pangeab-blip/EvGeo-Exercises/blob/main/CO2_Buildup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# ============================================================================
#  CO2 Atmosphere–Ocean Mini-Model (Colab, no-weathering) — con spegnimento
# ============================================================================
# Scopo operativo
#   Stimare: (i) il tempo necessario per raggiungere un target atmosferico A* (ppm)
#            con input vulcanico costante D; (ii) il tempo di rilassamento verso
#            l’equilibrio dopo lo spegnimento dell’input a t_off (scelto in vari modi).
#
# Variabili/parametri (unità tra parentesi)
#   f_atm_eq  (—)        Frazione di equilibrio dell’atmosfera: quota del carbonio totale
#                        che appartiene all’atmosfera a regime (0 < f < 1).
#   tau_ex    (yr)       Tempo caratteristico di scambio atmosfera–oceano.
#   D         (MtCO2/yr) Tasso di ingresso (degassamento) di CO2, costante finché non si spegne.
#   A0        (ppm)      Concentrazione atmosferica di partenza.
#   A_star    (ppm)      Target atmosferico desiderato.
#   dt        (yr)       Passo di integrazione numerica.
#   cutoff_mode          Modalità di spegnimento dell’input:
#                        - "hit_target": spegni quando A(t) raggiunge A*.
#                        - "eq_match":  spegni quando l’equilibrio finale sarà esattamente A*.
#                        - "manual":    spegni a t_off specificato dall’utente.
#   t_off     (yr)       Tempo di spegnimento manuale (usato solo se cutoff_mode="manual").
#
# Quantità derivate
#   D_ppm = D / 7800          Conversione input → ppm/yr (1 ppm CO2 ≈ 7.8 GtCO2 = 7800 MtCO2).
#   C_tot(t)                  CO2 totale accumulata (ppm-equivalenti) = ∫ D_ppm dt finché input acceso.
#   A_eq_final                Equilibrio finale dopo lo spegnimento: f * C_tot_final (ppm).
#
# Tempi diagnostici restituiti
#   t_hit          Tempo per “colpire” il target A* con input acceso (se avviene).
#   t_off_used     Istante effettivo di spegnimento utilizzato in simulazione.
#   t_to_95eq      Tempo dopo t_off per arrivare al 95% del salto verso A_eq_final (≈ 3 tau).
#   t_to_99eq      Tempo dopo t_off per arrivare al 99% del salto verso A_eq_final (≈ 4.6 tau).
#
# Equazioni del modello (due-box; no weathering)
#   dA/dt = D_ppm(t) - F_ex
#   dO/dt = + F_ex * 7.8                 (O in GtCO2)
#   F_ex  = (A - f * C_tot_ppm) / tau    (flusso A→O in ppm/yr)
#   C_tot_ppm = A + O/7.8
#
# Nota fisica chiave
#   In regime a input acceso, per t >> tau: A(t) ≈ f * D_ppm * t + offset.
#   Se spegni a t_off, il totale C_tot si fissa e A(t) rilassa esponenzialmente verso
#   A_eq_final = f * C_tot_final con tempo caratteristico tau.
# ============================================================================

import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Conversioni
PPM_TO_GtCO2 = 7.8
GtCO2_TO_PPM = 1.0 / PPM_TO_GtCO2
MT_PER_PPM   = 7800.0  # MtCO2 per 1 ppm CO2

# --- Utility di formattazione
def fmt_years(val):
    if val is None or np.isnan(val):
        return "n.d."
    if val >= 1e6:
        return f"{val/1e6:.3f} Myr"
    if val >= 1e4:
        return f"{val/1e3:.1f} kyr"
    return f"{val:.0f} yr"

# --- Funzioni base del modello
def step_AO(A, O_Gt, f, tau, D_ppm, dt):
    Ctot_ppm = A + O_Gt * GtCO2_TO_PPM
    A_eq = f * Ctot_ppm
    F_ex_ppm = (A - A_eq) / tau            # + = Atmosfera → Oceano
    A_new  = A + (D_ppm - F_ex_ppm) * dt
    O_newG = O_Gt + (F_ex_ppm * PPM_TO_GtCO2) * dt
    return A_new, O_newG

def simulate_with_cutoff(f, tau, D_mt_yr, A0_ppm, A_star_ppm,
                         cutoff_mode="hit_target", t_off_manual=1e5,
                         dt=1.0, t_tail_factor=12.0):
    D_ppm = D_mt_yr / MT_PER_PPM

    # Determina t_off secondo la modalità scelta
    t_off = None
    if cutoff_mode == "eq_match":
        # Equilibrio finale desiderato: A_eq_final = A_star = f * C_tot_final
        # Con input costante: C_tot_final = A0 + D_ppm * t_off  (A0 contribuisce già a C_tot)
        # => t_off = (A_star - f*A0) / (f*D_ppm)
        s = f * D_ppm
        t_off = np.inf if s <= 0 else max(0.0, (A_star_ppm - f*A0_ppm) / s)
    elif cutoff_mode == "manual":
        t_off = max(0.0, float(t_off_manual))
    else:
        # "hit_target": calcolato dinamicamente durante la simulazione quando A raggiunge A*
        t_off = None  # sarà determinato al volo

    # Stima tempo massimo di simulazione
    # coda dopo lo spegnimento: t_tail_factor * tau, con minimo ragionevole
    t_tail = max(t_tail_factor * tau, 10_000.0)
    if cutoff_mode in ("eq_match", "manual"):
        t_max = max(t_off + t_tail, 50_000.0)
    else:
        # per "hit_target" dimensiono progressivamente; prima un upper bound ottimistico
        t_max = 2.0 * (A_star_ppm / max(f * D_ppm, 1e-12)) + t_tail

    n = int(np.ceil(t_max / dt)) + 1
    t = np.linspace(0.0, t_max, n)
    A = np.zeros(n)
    O = np.zeros(n)
    A[0] = A0_ppm
    O[0] = 0.0

    t_hit = None
    t_off_used = None
    input_on = True

    for i in range(1, n):
        ti = t[i]

        # Se modalità "hit_target" e non abbiamo ancora spento,
        # spegni appena superi A_star.
        if cutoff_mode == "hit_target" and input_on and A[i-1] >= A_star_ppm and t_hit is None:
            t_hit = t[i-1]
            t_off_used = t_hit
            input_on = False

        # Se modalità eq_match/manual: l'input si spegne a t_off prefissato
        if cutoff_mode in ("eq_match", "manual") and input_on and ti >= t_off:
            t_off_used = t_off
            input_on = False

        D_in_ppm = D_ppm if input_on else 0.0
        A[i], O[i] = step_AO(A[i-1], O[i-1], f, tau, D_in_ppm, dt)

        # Registra t_hit nel caso si raggiunga A_star prima di spegnere
        if t_hit is None and A[i] >= A_star_ppm and input_on:
            t_hit = ti

    # Calcoli di equilibrio finale
    if t_off_used is None:
        # Se non abbiamo mai spento (può capitare se A* è irraggiungibile coi parametri scelti),
        # prendiamo come t_off_used la fine del dominio ma segnaliamo la condizione.
        t_off_used = t.max()

    # C_tot_final in ppm-equivalenti all'istante di spegnimento effettivo
    # (trova l’indice più vicino a t_off_used)
    i_off = int(np.clip(round(t_off_used / dt), 0, n-1))
    Ctot_final_ppm = A[i_off] + O[i_off] * GtCO2_TO_PPM
    A_eq_final = f * Ctot_final_ppm

    # Tempi per raggiungere frazioni del rilassamento dopo t_off_used
    # Definizione: A(t) = A_eq_final + (A(t_off) - A_eq_final) * exp(-(t - t_off)/tau)
    # Tempo per arrivare al 95%: |Δ(t)| = 0.05*|Δ0| => t_95 = tau * ln(1/0.05) ≈ 2.996 tau
    # Tempo per arrivare al 99%: |Δ(t)| = 0.01*|Δ0| => t_99 = tau * ln(1/0.01) ≈ 4.605 tau
    t_to_95eq = 2.9957 * tau
    t_to_99eq = 4.6052 * tau

    results = dict(
        t=t, A=A, O=O,
        t_hit=t_hit,
        t_off_used=t_off_used,
        A_eq_final=A_eq_final,
        Ctot_final_ppm=Ctot_final_ppm,
        t_to_95eq=t_to_95eq,
        t_to_99eq=t_to_99eq
    )
    return results

# --- Widget UI
defaults = dict(f=0.05, tau=5000.0, D=300.0, A0=280.0, Astar=500.0, dt=1.0,
                cutoff_mode="hit_target", t_off_manual=100_000.0)

f_dd   = widgets.FloatText(value=defaults["f"], description="f_atm_eq", step=0.01)
tau_dd = widgets.FloatText(value=defaults["tau"], description="tau_ex (yr)", step=50.0)
D_box  = widgets.FloatText(value=defaults["D"], description="D (Mt/yr)", step=10.0)
A0_box = widgets.FloatText(value=defaults["A0"], description="A0 (ppm)", step=5.0)
Astar  = widgets.FloatText(value=defaults["Astar"], description="Target A* (ppm)", step=10.0)
dt_box = widgets.FloatText(value=defaults["dt"], description="dt (yr)", step=0.5)

cutoff_mode_dd = widgets.Dropdown(
    options=[("Spegni quando A=target", "hit_target"),
             ("Spegni per avere A_eq_final=target", "eq_match"),
             ("Spegni a t_off manuale", "manual")],
    value=defaults["cutoff_mode"],
    description="Cutoff mode"
)
t_off_box = widgets.FloatText(value=defaults["t_off_manual"], description="t_off (yr)", step=1000.0)

reset_btn = widgets.Button(description="Reset", button_style="warning")
out = widgets.Output()

def run(_=None):
    with out:
        clear_output(wait=True)
        f   = max(1e-6, float(f_dd.value))
        tau = max(1e-6, float(tau_dd.value))
        D   = max(0.0,   float(D_box.value))
        A0  = max(0.0,   float(A0_box.value))
        A_  = max(0.0,   float(Astar.value))
        dt  = max(1e-3,  float(dt_box.value))
        mode = cutoff_mode_dd.value
        t_off_manual = float(t_off_box.value)

        res = simulate_with_cutoff(
            f=f, tau=tau, D_mt_yr=D, A0_ppm=A0, A_star_ppm=A_,
            cutoff_mode=mode, t_off_manual=t_off_manual, dt=dt
        )

        t, A, O = res["t"], res["A"], res["O"]
        t_hit       = res["t_hit"]
        t_off_used  = res["t_off_used"]
        A_eq_final  = res["A_eq_final"]
        Ctot_final  = res["Ctot_final_ppm"]
        t95, t99    = res["t_to_95eq"], res["t_to_99eq"]

        # Riassunto numerico
        df = pd.DataFrame([{
            "f_atm_eq": f,
            "tau_ex (yr)": tau,
            "D (Mt/yr)": D,
            "A0 (ppm)": A0,
            "Target A* (ppm)": A_,
            "Cutoff mode": mode,
            "t_hit (A=target)": (np.nan if t_hit is None else t_hit),
            "t_off_used (yr)": t_off_used,
            "A_eq_final (ppm)": A_eq_final,
            "C_tot_final (ppm-eq)": Ctot_final,
            "t_to_95% eq (yr)": t95,
            "t_to_99% eq (yr)": t99
        }])
        display(df.style.format({
            "t_hit (A=target)": fmt_years,
            "t_off_used (yr)": fmt_years,
            "A_eq_final (ppm)": "{:,.1f}",
            "C_tot_final (ppm-eq)": "{:,.1f}",
            "t_to_95% eq (yr)": fmt_years,
            "t_to_99% eq (yr)": fmt_years
        }))

        # Grafico 1: A(t) con spegnimento e linea di equilibrio finale
        plt.figure(figsize=(9,5.5))
        plt.plot(t, A, label="A(t) atmosfera (ppm)")
        plt.axhline(A_eq_final, linestyle="--", label=f"A_eq_final = {A_eq_final:.1f} ppm")
        if t_hit is not None:
            plt.axvline(t_hit, linestyle=":", label=f"t_hit (A=target) ≈ {fmt_years(t_hit)}")
        if t_off_used is not None:
            plt.axvline(t_off_used, linestyle="-.", label=f"t_off ≈ {fmt_years(t_off_used)}")
            plt.axvline(t_off_used + t95, linestyle=":", color='gray', label=f"+95% eq ≈ {fmt_years(t95)}")
            plt.axvline(t_off_used + t99, linestyle=":", color='black', label=f"+99% eq ≈ {fmt_years(t99)}")
        plt.axhline(A_, linestyle=":", color='tab:green', label=f"Target A* = {A_:.0f} ppm")
        plt.xlabel("Time (years)")
        plt.ylabel("Atmospheric CO$_2$ (ppm)")
        plt.title("Raggiungimento del target e rilassamento verso l’equilibrio (input con spegnimento)")
        plt.grid(True, linestyle=":")
        plt.legend()
        plt.show()

        # Grafico 2: partizione cumulativa (ppm-equivalenti)
        O_ppm = O * GtCO2_TO_PPM
        plt.figure(figsize=(9,5.5))
        plt.plot(t, A,     label="Atmosfera (ppm)")
        plt.plot(t, O_ppm, label="Oceano (ppm-equivalenti)")
        if t_off_used is not None:
            plt.axvline(t_off_used, linestyle="-.", label="t_off")
        plt.xlabel("Time (years)")
        plt.ylabel("CO$_2$ (ppm-equivalent)")
        plt.title("Partizione cumulativa CO$_2$: Atmosfera vs Oceano")
        plt.grid(True, linestyle=":")
        plt.legend()
        plt.show()

def do_reset(_):
    f_dd.value   = defaults["f"]
    tau_dd.value = defaults["tau"]
    D_box.value  = defaults["D"]
    A0_box.value = defaults["A0"]
    Astar.value  = defaults["Astar"]
    dt_box.value = defaults["dt"]
    cutoff_mode_dd.value = defaults["cutoff_mode"]
    t_off_box.value = defaults["t_off_manual"]
    run()

# Layout UI
controls_row1 = widgets.HBox([D_box, Astar, A0_box, dt_box])
controls_row2 = widgets.HBox([f_dd, tau_dd])
controls_row3 = widgets.HBox([cutoff_mode_dd, t_off_box, reset_btn])
ui = widgets.VBox([controls_row1, controls_row2, controls_row3, out])

reset_btn.on_click(do_reset)

display(ui)
run()

for w in [f_dd, tau_dd, D_box, A0_box, Astar, dt_box, cutoff_mode_dd, t_off_box]:
    w.observe(run, names="value")